In [ ]:
!git clone https://github.com/sskyisthelimit/DS_Internship.git

Cloning into 'DS_Internship'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 119 (delta 47), reused 106 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (119/119), 6.32 MiB | 10.27 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [ ]:
cd DS_Internship/NLP/

/content/DS_Internship/NLP


In [ ]:
!pip install -r requirements.txt

In [ ]:
cd src

/content/DS_Internship/NLP/src


Simple example, check hard test examples, classification reports below.

Note: I don't think whether label is MOUNTAIN or ELEVATION is relevant, so in
classification report if B-ELEVATION is predicted instead of B-MOUNTAIN
it is considered still correct (or I-ELEVATION is predicted instead of I-MOUNTAIN) check everything by yourself carefully, I don't think you can call this cheating etc. considering given task.

After thinking about it i came to conclusion that it might be called cheating, so i also provided report
considering mountain and elevation 2 separate classes but i'm sure other contestants didn't do mountain/elevation separations. I think i could have drastically improved elevation/mount distinguishing by writing custom loss with focus on distinguishing them.

In [ ]:
!python -m inference --sentence "Mid year the plan is to do the Eiger south face." --device "cpu"

tokenizer_config.json: 100% 1.19k/1.19k [00:00<00:00, 5.43MB/s]
vocab.txt: 100% 213k/213k [00:00<00:00, 1.34MB/s]
tokenizer.json: 100% 669k/669k [00:00<00:00, 2.72MB/s]
special_tokens_map.json: 100% 125/125 [00:00<00:00, 687kB/s]
config.json: 100% 919/919 [00:00<00:00, 4.76MB/s]
model.safetensors: 100% 431M/431M [00:10<00:00, 42.4MB/s]
Mid: O
year: O
the: O
plan: O
is: O
to: O
do: O
the: O
Eiger: B-ELEVATION
south: O
face: O
.: O


I tried to make test set as challenging and real world as possible by picking sentences from reddit r/alpinism

In [ ]:
import json
# loading test dataset from repo
with open("../datasets/test_dataset.json", "r") as f:
  test_samples = json.load(f)

print(json.dumps(test_samples))

{"classes": ["B-MOUNTAIN", "B-ELEVATION", "I-MOUNTAIN", "I-ELEVATION"], "annotations": [["No im not asking for your secret spot, I\u2019m heading to Tahoe for some bc skiing in a month and I\u2019m hoping to get some ice climbing in.", {"entities": [[54, 59, "B-MOUNTAIN"]]}], ["-15\u00b0c is generally the coldest I go climb at due to the ice fragility. I use Batura 2.0, not the warmest but probably warmer than the Mont Blanc.", {"entities": []}], ["Have winter climbed in Scotland before with Mont Blanc Pros but it seems like Japan is gonna be somewhat colder than what I have experienced there.", {"entities": []}], ["Hello, i\u2019m going to climb the Aneto (Pyrenees) in winter, are boots like the Salewa Crow enough? Or will my feet get cold?", {"entities": [[30, 35, "B-MOUNTAIN"], [37, 45, "B-ELEVATION"]]}], ["I live in New Brunswick, Canada, and want to pick up a somewhat versatile tool that I can use for easy waterfall ice, but also rely on for climbs on Katahdin and occasionally the

In [ ]:
from inference import (model, tokenizer, post_process_tokens_and_labels,
                       predict_ner_labels, labels, label2id, id2label)

In [ ]:
import re
from utils import convert_to_bert_format

device = "cpu"
# Converting samples to same format as outputs of my model
# code at the end of src/utils.py
formatted_test_samples = []
for sample in test_samples["annotations"]:
  formatted_test_samples.append(
      convert_to_bert_format(sample[0], sample[1]["entities"], tokenizer))


In [ ]:
print(formatted_test_samples[0][0])
print(formatted_test_samples[0][1])
print(formatted_test_samples[0][2])

['No', 'im', 'not', 'asking', 'for', 'your', 'secret', 'spot', ',', 'I', '’', 'm', 'heading', 'to', 'Ta', '##hoe', 'for', 'some', 'b', '##c', 'skiing', 'in', 'a', 'month', 'and', 'I', '’', 'm', 'hoping', 'to', 'get', 'some', 'ice', 'climbing', 'in', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
No im not asking for your secret spot, I’m heading to Tahoe for some bc skiing in a month and I’m hoping to get some ice climbing in.


Main focus of improving model should be on improving recall,sampling real sentences definitely can help, and some type of custom loss that will help with improvment of recall, maybe deleting all non-mount sentences from dataset also could help (last - not sure).

In [ ]:
import torch
from sklearn.metrics import classification_report
import spacy
from spacy.tokens import Doc, Span
from spacy import displacy

# SpaCy setup for visualization
nlp = spacy.blank("en")  # Create a blank SpaCy pipeline

def create_spacy_doc(tokens, labels, label_map):
    """
    Create a SpaCy Doc object with labeled entities for visualization.

    Args:
    tokens (list): List of tokens.
    labels (list): List of corresponding labels.
    label_map (dict): Mapping from label IDs to label names.

    Returns:
    Doc: A SpaCy Doc object with labeled entities.
    """
    doc = Doc(nlp.vocab, words=tokens)
    entities = []
    for i, label in enumerate(labels):
        if label > 0:  # Assuming 0 is 'O' or non-entity
            entities.append(Span(doc, i, i + 1, label=label_map[label]))
    doc.ents = entities
    return doc



def compute_metrics_and_visualize(formatted_test_samples, model, tokenizer, device, label_map):
    """
    Compute metrics and visualize problematic sentences.

    Args:
    formatted_test_samples (list): List of tuples containing tokens, labels, and sentences.
    model (torch.nn.Module): The fine-tuned NER model.
    tokenizer (transformers.PreTrainedTokenizer): The tokenizer used to tokenize the input.
    device (torch.device): The device (CPU or GPU) on which to run the model.
    label_map (dict): Mapping from label IDs to label names.
    """
    y_true, y_pred = [], []

    for tokens, labels, sentence in formatted_test_samples:
        # Predict labels for the sentence
        pred_tokens, pred_labels = predict_ner_labels(sentence, model, tokenizer, device, False)
        tokens, labels = post_process_tokens_and_labels(tokens, labels)

        # Adjust labels for equivalence rules
        adjusted_preds = [
            true if (label == true or (label, true) in [(3, 1), (1, 3), (2, 4), (4, 2)]) else 0
            for label, true in zip(pred_labels, labels)
        ]

        y_true.extend(labels)
        y_pred.extend(adjusted_preds)

        # Check for problematic sentences
        if labels != adjusted_preds:
            print("Problematic Sentence:")
            print("Original:")
            displacy.render(create_spacy_doc(tokens, labels, label_map), style="ent", jupyter=True)
            print("Predicted:")
            displacy.render(create_spacy_doc(pred_tokens, adjusted_preds, label_map), style="ent", jupyter=True)

    # Compute metrics
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=list(label_map.values())))


compute_metrics_and_visualize(formatted_test_samples, model, tokenizer, device, id2label)


Problematic Sentence:
Original:


Predicted:


Problematic Sentence:
Original:


Predicted:


Problematic Sentence:
Original:


...

Problematic Sentence:
Original:


Predicted:


Problematic Sentence:
Original:


Predicted:


Classification Report:
              precision    recall  f1-score   support

           O       0.98      1.00      0.99      1040
  B-MOUNTAIN       1.00      0.85      0.92        55
  I-MOUNTAIN       1.00      0.92      0.96        38
 B-ELEVATION       1.00      0.54      0.70        13
 I-ELEVATION       1.00      0.67      0.80         6

    accuracy                           0.98      1152
   macro avg       1.00      0.80      0.87      1152
weighted avg       0.98      0.98      0.98      1152



False negatives seem to be grouped, especially when person talks about 'doing' some mountain or peak - in terms of planning future climbs.

Also model struggles with sentences where person explains future plans for multiple mountains or some specific peak of mountain.

In [ ]:
def compute_metrics_and_visualize(formatted_test_samples, model, tokenizer, device, label_map):
    """
    Compute metrics and visualize problematic sentences.

    Args:
    formatted_test_samples (list): List of tuples containing tokens, labels, and sentences.
    model (torch.nn.Module): The fine-tuned NER model.
    tokenizer (transformers.PreTrainedTokenizer): The tokenizer used to tokenize the input.
    device (torch.device): The device (CPU or GPU) on which to run the model.
    label_map (dict): Mapping from label IDs to label names.
    """
    y_true, y_pred = [], []

    for tokens, labels, sentence in formatted_test_samples:
        # Predict labels for the sentence
        pred_tokens, pred_labels = predict_ner_labels(sentence, model, tokenizer, device, False)
        tokens, labels = post_process_tokens_and_labels(tokens, labels)

        # Adjust labels for equivalence rules
        adjusted_preds = [
            label if (label == true or (label, true) in [(3, 1), (1, 3), (2, 4), (4, 2)]) else 0
            for label, true in zip(pred_labels, labels)
        ]

        y_true.extend(labels)
        y_pred.extend(adjusted_preds)

        # Check for problematic sentences
        if labels != adjusted_preds:
            print("Problematic Sentence:")
            print("Original:")
            displacy.render(create_spacy_doc(tokens, labels, label_map), style="ent", jupyter=True)
            print("Predicted:")
            displacy.render(create_spacy_doc(pred_tokens, adjusted_preds, label_map), style="ent", jupyter=True)

    # Compute metrics
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=list(label_map.values())))


compute_metrics_and_visualize(formatted_test_samples, model, tokenizer, device, id2label)

Problematic Sentence:
Original:


Predicted:


Problematic Sentence:
Original:


Predicted:


Problematic Sentence:
Original:


...

Problematic Sentence:
Original:


Predicted:


Problematic Sentence:
Original:


Predicted:


Classification Report:
              precision    recall  f1-score   support

           O       0.98      1.00      0.99      1040
  B-MOUNTAIN       0.97      0.56      0.71        55
  I-MOUNTAIN       1.00      0.68      0.81        38
 B-ELEVATION       0.27      0.46      0.34        13
 I-ELEVATION       0.31      0.67      0.42         6

    accuracy                           0.96      1152
   macro avg       0.71      0.68      0.66      1152
weighted avg       0.97      0.96      0.96      1152



this task i more sophisticated, we need more class distinguishing specific data and some custom loss.